In [249]:
import json

sents = []
iob_tags = []
text = ""
filename = "./re3d/train.txt"
# filename = './conll2003/small_data.txt'
with open(filename, 'r') as file:
    lst = []
    while True:
        word_lst = file.readline().lower().split('\t')
        word = word_lst[0]
        if not word:
            break
        text += " " + word

        if word == '\n':
            sents.append(lst)
            lst = []
        else:
            lst.append(word)
            iob_tags.append(word_lst[-1].split('\n')[0])

    sents.append(lst)

with open('./re3d/sentences.txt', 'w') as f:
    f.write(text)
    f.close()

### Distinct Types of IOB Tokens

In [250]:
dist = set()
for tok in iob_tags:
    dist.add(tok)

print(dist)

{'i-weapon', 'b-documentreference', 'i-militaryplatform', 'b-militaryplatform', 'b-weapon', 'b-money', 'b-temporal', 'i-location', 'b-nationality', 'b-organisation', 'i-temporal', 'o', 'i-quantity', 'b-quantity', 'i-person', 'i-money', 'i-organisation', 'i-documentreference', 'b-person', 'i-nationality', 'b-location'}


In [251]:
# Here we are checking and ensuring that both number of tokens in sentences and number of tags in iob_tags are same
print('Number of iob tags are: ',len(iob_tags))

l=0
for i in range(len(sents)):
    l+=len(sents[i])

print('Number of tokens are: ',l)

Number of iob tags are:  20030
Number of tokens are:  20030


In [235]:
# converting iob tags to numerical tags
o = 0
b_org = 1
i_org = 2
b_per = 3
i_per = 4
b_loc = 5
i_loc = 6
b_misc = 7
i_misc = 8

iob_dict = {'o': 0, 'b-org' : 1, 'i-org' : 2, 'b-per' : 3, 'i-per' : 4, 'b-loc' : 5, 'i-loc' : 6, 'b-misc' : 7, 'i-misc' : 8}

num_iob_tags = []
for tok in iob_tags:
    num_iob_tags.append(iob_dict[tok])

num_iob_tags = np.array(num_iob_tags)

In [225]:
import numpy as np
from scipy.sparse import dok_matrix, vstack, csr_matrix
import random



# helper functions
def dict_match(sentence, dictionary, max_ngrams=4):
   m = {}
   for i in range(len(sentence)):
       for j in range(i+1, min(len(sentence), i + max_ngrams) + 1):
           term = ' '.join(sentence[i:j])
           if term in dictionary:
               m.update({idx:1 for idx in range(i,j+1)})
   return m
           
         
def create_token_L_mat(Xs, Ls, num_lfs):
   """
   Create token-level LF matrix from LFs indexed by sentence
   """
   Yws = []
   for sent_i in range(len(Xs)):
       ys = dok_matrix((len(Xs[sent_i]), num_lfs))
       for lf_i in range(num_lfs):
           for word_i,y in Ls[sent_i][lf_i].items():
               ys[word_i, lf_i] = y
       Yws.append(ys)
   return csr_matrix(vstack(Yws))


  
# labeling functions
def LF_is_org(s):
   dict = {}
   if s[0]== 'the' and len(s)>2:
      dict[1] = b_org

   return dict




def LF_is_per(s):
   dict = {}
   # for i in range(len(s)):
   #    p = random.random()
   #    if p<=0.5:
   #       dict[i] = B_PER
   return dict



def LF_is_loc(s):
   dict = {}
   # for i in range(len(s)):
   #    p = random.random()
   #    if p>=0.5:
   #       dict[i] = I_ORG
   return dict



lfs = [
   LF_is_org,
   LF_is_per,
   LF_is_loc
]

# apply labeling functions and transform label matrix 
L = [[lf(s) for lf in lfs] for s in sents] 

L = create_token_L_mat(sents, L, len(lfs))  
L = np.asarray(L.astype(np.int8).todense())

In [226]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=4, verbose=True)
label_model.fit(L_train=L, n_epochs=500, log_freq=100, seed=123)

label_model.predict(L)


INFO:root:Computing O...
INFO:root:Estimating \mu...
100%|██████████| 500/500 [00:00<00:00, 515.85epoch/s]
INFO:root:Finished Training


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0])

In [241]:
# finding accuracy
label_model_acc = label_model.score(L=L, Y=num_iob_tags, tie_break_policy="random")["accuracy"]
print(label_model_acc)

0.8452830188679246
